In [ ]:
import bubblebox as bb
from bubblebox.mdbox import lj_force, coulomb_force, mdbox
import numpy as np

In [ ]:
b = bb.mdbox(7, size= (-2.5,-2.5))

b.set_forces(lj_force, force_params=np.array([50,2.5*2**(-1/6)]))

#pick N random spheres which repel

#ba = np.random.choice(b.n_bubbles, 3, replace = False)
#b.set_forces(coulomb_force, bubbles_a=ba, bubbles_b = ba, force_params = np.array([6,6]))

b.set_masses(np.random.randint(1,4,b.masses.shape[0]))
b.view()

In [ ]:
for i in range(10000):
    for j in range(3):
        b.advance()
    b.vel_ *= .999
    b.update_view()

In [ ]:
def extend_system(b, n_layers = 1):
    """
    add n_layers extra layers around the simulation cell, cloning content in the reference
    """
    
    lattice_vec = 2*np.abs(np.array(b.size))*np.eye(len(b.size))
    lattice_coords = np.array(np.meshgrid(*[np.arange(-1,1+1) for i in range(len(b.size))])).reshape( len(b.size), -1)
    
    
    
    n_cells = 3**len(b.size) 
    nb = b.n_bubbles*n_cells
    
    B = mdbox(nb, size = np.array(b.size)*3)
    
    
    pos = np.zeros((len(b.size), nb ), dtype = float)
    masses = np.zeros(nb, dtype = float)
    
    print(b.pos.shape)
    
    for i in range(n_cells):
        pos[:, i*b.n_bubbles:(i+1)*b.n_bubbles] = b.pos + lattice_coords[:,i].dot(lattice_vec)[:, None] 
        #print("masses:", masses[i*b.n_bubbles:(i+1)*b.n_bubbles])
        masses[i*b.n_bubbles:(i+1)*b.n_bubbles] = b.masses
        for j in range(n_cells):
            B.interactions[i*b.n_bubbles:(i+1)*b.n_bubbles, j*b.n_bubbles:(j+1)*b.n_bubbles] = b.interactions
        
    B.pos = pos
    B.set_masses(masses)
     
    return B

    
    
    
B = extend_system(b)

B.view()

In [ ]:
B.vel_ *= 10.4
for i in range(10000):
    for j in range(3):
        B.advance()
    #B.vel_ *= .999
    B.update_view()

In [ ]:
B.evince()